<a href="https://colab.research.google.com/github/dimlish/StarAlgorithm/blob/main/Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets


%matplotlib inline

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 8

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.c1 = nn.Conv2d(3, 12,  4)
        self.c2 = nn.Conv2d(12, 36, 4)
        self.l1 = nn.Linear(900, 200)
        self.l2 = nn.Linear(200, 50)
        self.l3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.c1(x)))
        x = self.pool(F.relu(self.c2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = nn.Softmax(dim=-1)(self.l3(x))
        return x

In [ ]:
class ModelStar(nn.Module):
    def __init__(self, model1):
        super().__init__()
        self.model1 = model1
        for p in self.model1.parameters():
            p.requires_grad = False
        self.pool = nn.MaxPool2d(2, 2)
        self.c1 = nn.Conv2d(3, 12, 4)
        self.c2 = nn.Conv2d(12, 36, 4)
        self.l1 = nn.Linear(900, 200)
        self.l2 = nn.Linear(200, 50)
        self.l3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.c1(x)))
        x = self.pool(F.relu(self.c2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = nn.Softmax(dim=-1)(self.l3(x))
        return x

In [4]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.c11 = nn.Conv2d(3, 12, 4)
        self.c12 = nn.Conv2d(12, 36, 4)
        self.l11 = nn.Linear(900, 200)
        self.l12 = nn.Linear(200, 50)
        self.l13 = nn.Linear(50, 10)

    def forward(self, x):
        x1 = x.copy()
        x2 = x.copy()
        x1 = self.pool(F.relu(self.c11(x)))
        x1 = self.pool(F.relu(self.c12(x)))
        x1 = torch.flatten(x, 1)
        x1 = F.relu(self.l11(x))
        x1 = F.relu(self.l12(x))
        x1 = nn.Softmax(dim=-1)(self.l13(x))
        return x

In [6]:
base1 = Model1()
crit = nn.CrossEntropyLoss()
optimizer = optim.SGD(base1.parameters(), lr=0.01)
base1.to(device)

Model1(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c1): Conv2d(3, 12, kernel_size=(4, 4), stride=(1, 1))
  (c2): Conv2d(12, 36, kernel_size=(4, 4), stride=(1, 1))
  (l1): Linear(in_features=900, out_features=200, bias=True)
  (l2): Linear(in_features=200, out_features=50, bias=True)
  (l3): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
for epoch in range(30):
    for x_train, y_train in train_loader:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        y_pred = base1(x_train)
        loss = crit(y_pred, y_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    if epoch % 2 == 0:
        mean_val_loss = []
        val_accuracy = []
        with torch.no_grad():
            for x_test, y_test in test_loader:
                x_test = x_test.to(device)
                y_test = y_test.to(device)
                y_pred = base1(x_test)
                loss = crit(y_pred, y_test)
                mean_val_loss.append(loss.numpy())
                val_accuracy.extend((torch.argmax(y_pred, dim=-1) == y_test).numpy().tolist())
        print('Epoch: {epoch}, loss: {loss}, accuracy: {accuracy}'.format(
                epoch=epoch, loss=np.mean(mean_val_loss), accuracy=np.mean(val_accuracy)
        ))
    

Epoch: 0, loss: 1.9202008247375488, accuracy: 0.5372
Epoch: 2, loss: 1.9159451723098755, accuracy: 0.541
Epoch: 4, loss: 1.8786888122558594, accuracy: 0.5819
Epoch: 6, loss: 1.858500361442566, accuracy: 0.6052
Epoch: 8, loss: 1.835591197013855, accuracy: 0.6237
